# Scribble visualisation:

#### This Notebook is a visualisation tool:
#### With this notebook you can:
#### - Visualize pathologist annotations
#### - Create scribbles for healthy regions and metastatic regions
#### - Send pathologist annotations and scribbles annotations to cytomine

In [1]:
import sys
from pathlib import Path
sys.path.append('/home/ahabis/2-Scribble_project')

### Libraries to import 

In [2]:
from config import *
from utils import *
from cytomine.models import ProjectCollection
from cytomine import Cytomine
from cytomine.models import CurrentUser
from send_annotation import *
from dell_annotation import delete_annotations
from send_annotation import Send
from scribble_inside_shape import Scribble
import warnings
import numpy as np
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')
pb_key = os.environ['PB_KEY']
pv_key = os.environ['PV_KEY']
host = 'https://nsclc.cytomine.com'

In [3]:
import pandas as pd
dataframe_test = pd.read_csv(path_dataframe_test,index_col = 0)
dataframe_test.shape


(60494, 3)

### Image Selection

In [4]:
### CHOOSE THE IMAGE YOU WANT TO WOR WITH 
filename = 'test_046'
#### Write if the image comes from train or test set
split = 'test'
###### CHECK THE ID on CYTOMINE
project = 10355395
image = 16770152
####
if split == 'train':
    path_image = os.path.join(path_slide_tumor_train,filename+'.tif')
else:
    path_image = os.path.join(path_slide_tumor_test,filename+'.tif')

In [5]:
img = OpenSlide(path_image)

### Visualisation of the Scribble

In [6]:
s = Scribble(filename,
             percent=0.0,
             show = True,
             split =split)
dataframe_annotation = s.create_dataframe_annotations()

In [7]:
# x = 0

# # for filename in os.listdir(path_slide_tumor_train):
#     # print(filename)
# s = Scribble(filename.split('.')[0],
#             percent=0.0,
#             show = True,
#             split =split)
# dataframe_annotation = s.create_dataframe_annotations()
# x += np.minimum(dataframe_annotation.shape[-1],10)


In [8]:
def visualize_scribble(dataframe_annotation,annotation_id):

    size1 = 15
    size2 = 12
    
    ann = dataframe_annotation[annotation_id]
    ann = ann[~ann.isnull()]
    ann_not_null = np.stack(ann)
    top = ann_not_null[:10]
    ann = np.concatenate([ann_not_null, top],0)


    plt.plot(ann[:,0], ann[:,1],color='orange',label='original contour')
    plt.legend(loc=3, prop={'size': size1})
    plt.axis('off')

    plt.show()
    arr, contour, scribble,coordinates = s.scribble(ann_not_null)

    plt.plot(contour[:,0], contour[:,1])
    plt.scatter(coordinates[:,0], coordinates[:,1],color='k', label='sampled points')
    plt.axis('off')

        

In [16]:
# visualize_scribble(dataframe_annotation,annotation_id)
print(filename)

test_046


### This code will get the pathologist's annotations, generate scribbles and send them to cytomine

In [9]:
contour_tumor, scribble_tumor, contour_healthy, scribble_healthy = get_scribbles_and_annotations(filename+'.tif', 'test')

HistolabException: NotADirectoryError(20, 'Not a directory'). This slide may be corrupted or have a non-standard format not handled by the openslide and PIL libraries. Consider setting use_largeimage to True when instantiating this Slide.

In [14]:
S = Send(filename,split='test')
S.send_annotations_to_cytomine(shapes,
                               scribbles,
                               contour_foreground,
                               scribble_foreground,
                               id_image=image,
                               id_project=project)

OpenSlideUnsupportedFormatError: Unsupported or missing image file